In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cleannotebook as cn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
import re
import optuna
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

# Set the working directory
os.chdir('C:/Users/julia/Documents/titanic')

AttributeError: partially initialized module 'pandas' from 'c:\Users\julia\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\__init__.py' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [ ]:
# get data into dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
ticket_lengths = []
for i in train_df.Ticket:
    len_ticket = len(i)
    ticket_lengths.append(len_ticket)
ticket_lengths = list(set(ticket_lengths))
print(ticket_lengths)

print(train_df[train_df.Ticket.apply(len) == 8])

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18]
     PassengerId  Survived  Pclass  \
1              2         1       1   
30            31         0       1   
31            32         1       1   
34            35         0       1   
52            53         1       1   
..           ...       ...     ...   
835          836         1       1   
846          847         0       3   
853          854         1       1   
863          864         0       3   
867          868         0       1   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
30                            Uruchurtu, Don. Manuel E    male  40.0      0   
31      Spencer, Mrs. William Augustus (Marie Eugenie)  female   NaN      1   
34                             Meyer, Mr. Edgar Joseph    male  28.0      1   
52            Harper, Mrs. Henry Sleeper (Myna Haxtun)  female  49.0      1   
..             

In [ ]:
X = train_df.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
y = train_df['Survived']
X_submit = test_df.drop(['Name', 'Ticket', 'PassengerId'], axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S


In [ ]:
# find unique titles from name 
def find_unique_titles(col):
    titles = []
    for i in col:
        result = re.search(r',\s([A-Za-z\s]+)\.', i)
        if result:
            titles.append(result.group(1))
    return list(set(titles))

# get titles from Name column into new column
def get_titles(df):
    titles = []
    for i in df.Name:
        result = re.search(r',\s([A-Za-z\s]+)\.', i)
        if result:
            titles.append(result.group(1))
        else:
            titles.append(np.nan)
    return titles
    

X['Title'] = get_titles(train_df)
X_submit['Title'] = get_titles(test_df)

# check number of nan values for Title -> zero na values
# X.loc[X['Title'].isna()]

In [ ]:
# women and children are more likely to survive -> extra column with boolean if the person is a woman or a child
# Master is a title used for little boys
for df in [X, X_submit]:
    df['IsFemaleOrBoy'] = ((df.Title == 'Master') | (df.Sex == 'female'))


In [ ]:
# X = cn.phase1clean(train_df)
# y = train_df['Survived']
# X = X[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'Deck', 'Family_Size']]
# X


In [ ]:
# change cabin number to deck letter
# X.Cabin.unique()
def cabin_to_deck(str):
    try:
        result = re.match(r'([A-Z]).*', str)
        if result:
            return result.group(1)
        else:
            return np.nan
    except:
        return np.nan
for df in [X, X_submit]:
    df['Deck']=df['Cabin'].map(lambda x: cabin_to_deck(x))
    df.drop('Cabin', axis=1, inplace=True)

    #Creating new family_size column
    df['Family_Size']=df['SibSp']+df['Parch']
    df.drop('SibSp', axis=1, inplace=True)
    df.drop('Parch', axis=1, inplace=True)

In [ ]:
# get fare per person
for df in [X, X_submit]:
    df['Fare_Per_Person']=df['Fare']/(df['Family_Size']+1)
    df.drop('Fare', axis=1, inplace=True)

In [ ]:
# plot all features
# for df in [X, X_submit]:
#     # df.plot(kind='box', figsize=(8,6))
#     # remove outliers
#     cols = ['Family_Size', 'Pclass', 'Age', 'Fare_Per_Person']
#     df[cols] = df[cols].clip(lower= df[cols].quantile(0.15), upper= df[cols].quantile(0.85), axis=1)
# #     df.plot(kind='box', figsize=(8,6))

In [ ]:
# define categorical features
def def_category_features(feature_list, df):
    for i in feature_list:
        df[i] = df[i].astype('category')

    return df

def_category_features(['Title', 'Sex', 'Deck', 'Embarked'], X)
def_category_features(['Title', 'Sex', 'Deck', 'Embarked'], X_submit)
# def_category_features(['Sex', 'Embarked', 'Title', 'Deck'], X)


,Pclass,Sex,Age,Embarked,Title,IsFemaleOrBoy,Deck,Family_Size,Fare_Per_Person
0,3,male,34.5,Q,Mr,False,NaN,0,7.829200
1,3,female,47.0,S,Mrs,True,NaN,1,3.500000
2,2,male,62.0,Q,Mr,False,NaN,0,9.687500
3,3,male,27.0,S,Mr,False,NaN,0,8.662500
4,3,female,22.0,S,Mrs,True,NaN,2,4.095833
...,...,...,...,...,...,...,...,...,...
413,3,male,NaN,S,Mr,False,NaN,0,8.050000
414,1,female,39.0,C,Dona,True,C,0,108.900000
415,3,male,38.5,S,Mr,False,NaN,0,7.250000
416,3,male,NaN,S,Mr,False,NaN,0,8.050000


In [ ]:
# get best parameters for lgbm classifier
def get_best_params_lgbm():
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.8, random_state=42
    )

    def objective(trial):
        params = {
            "num_leaves": trial.suggest_int("num_leaves", 20, 100),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
            "random_state": 42
        }
        
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        
        return acc  # maximize

    # start study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    print("Best parameters:")
    print(study.best_params)

    print("Best accuracy:", study.best_value)

In [ ]:
def lgbm_train_model(X,y):       
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                              train_size=0.8,
                                              random_state=42)
    
    # define model -> use lightgbm classifier with the parameters detemined before that showed the best accurcy
    model = LGBMClassifier(num_leaves=24, max_depth=6, learning_rate=0.02433205518116897, n_estimators=1523, min_child_samples=43, subsample=0.9397755948883224, colsample_bytree=0.7806928527472858, reg_alpha=0.7021034007342188,reg_lambda0=0.8046256149930574)
    # train model
    model.fit(X_train, y_train)

    # test model
    y_pred = model.predict(X_test)

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))

    print("Used features:")
    print(model.feature_name_)

    return model


In [ ]:
# get_best_params_lgbm()
model = lgbm_train_model(X,y)


[LightGBM] [Warning] Unknown parameter: reg_lambda0
[LightGBM] [Warning] Unknown parameter: reg_lambda0
[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 217
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

In [ ]:
# train lgbm classifier with optimal parameters and all training data
# predict test data and save to .csv file to submit
def lgbm_model_submit(X,y,X_submit):       
    # define model -> use lightgbm classifier with the parameters detemined before that showed the best accurcy
    model = LGBMClassifier(num_leaves=24, max_depth=6, learning_rate=0.02433205518116897, n_estimators=1523, min_child_samples=43, subsample=0.9397755948883224, colsample_bytree=0.7806928527472858, reg_alpha=0.7021034007342188,reg_lambda0=0.8046256149930574)
    # train model
    model.fit(X, y)

    # test model
    y_pred = model.predict(X_submit)
    output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': y_pred})
    output.to_csv('submission.csv', index=False)

    print('Submission successfully saved')

lgbm_model_submit(X,y,X_submit)

[LightGBM] [Warning] Unknown parameter: reg_lambda0
[LightGBM] [Warning] Unknown parameter: reg_lambda0
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 331
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def randomforest_train_model(X,y):    
    # random forest does not handle categorical features
    # use get_dummies function to convert categorical variable into dummy/indicator vaiables   
    features = ["Pclass", "Sex", "SibSp", "Parch"]
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                              train_size=0.8,
                                              random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    # train model
    model.fit(X_train, y_train)

    # test model
    y_pred = model.predict(X_test)

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))

    print("Used features:")
    print(model.feature_name_)

    return model

# randomforest_train_model(X,y)

# y = train_data["Survived"]

# features = ["Pclass", "Sex", "SibSp", "Parch"]
# X = pd.get_dummies(train_data[features])
# X_test = pd.get_dummies(test_data[features])
